In [ ]:
import cv2
from skimage.measure import compare_ssim
import numpy as np

def Compare(image1, image2):
    
    difference = cv2.subtract(image1, image2)

    Conv_hsv_Gray = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(Conv_hsv_Gray, 0, 255,cv2.THRESH_BINARY_INV |cv2.THRESH_OTSU)
    difference[mask != 255] = [0, 0, 255]

    image1[mask != 255] = [0, 0, 255]
    image2[mask != 255] = [0, 0, 255]
        
    cv2.imwrite('diffOverImage1.png', image1)
    cv2.imwrite('diffOverImage2.png', image2)
    cv2.imwrite('diff.png', difference)

    before = image1
    after = image2

    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

    # Compute SSIM between two images
    (score, diff) = compare_ssim(before_gray, after_gray, full=True)
    #print("Image similarity", score)

    diff = (diff * 255).astype("uint8")

    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    mask = np.zeros(before.shape, dtype='uint8')
    filled_after = after.copy()

    for c in contours:
        area = cv2.contourArea(c)
        if area > 40:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
            cv2.drawContours(mask, [c], 0, (0,255,0), -1)
            cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

    cv2.imshow('before', before)
    out0.write(before)
    
    cv2.imshow('after', after)
    out1.write(after)
    
#     cv2.imshow('diff',diff)
#     cv2.imshow('mask',mask)
#     cv2.imshow('filled after',filled_after)
    cv2.waitKey(1)

cap0 = cv2.VideoCapture('test_video0.avi')
cap1 = cv2.VideoCapture('test_video1.avi')

width0 = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height0 = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size0 = (width0, height0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out0 = cv2.VideoWriter('your_video0.avi', fourcc, 20.0, size0)

width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size1 = (width1, height1)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out1 = cv2.VideoWriter('your_video1.avi', fourcc, 20.0, size1)

while(True):
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    
    if (ret0):
        #display the resulting frame
        cv2.imshow('frame0', frame0)
    
    if (ret1):
        #display the resulting frame
        cv2.imshow('frame1', frame1)

    if ret0 != ret1:
        break
    
    Compare(frame0, frame1)
    k = cv2.waitKey(1)
    
    if k == ord('q'):
        break

cap0.release()
cap1.release()
cv2.destroyAllWindows()